In [23]:
import pandas as pd
from pymongo import MongoClient
import re
from haversine import haversine
client = MongoClient()
import function as f
import time

# Bring documents from Mongodb

In [2]:
db, coll = f.connectCollection('companies','companies')

In [3]:
compas = list(coll.find({"$and":[{"deadpooled_year":None},{"founded_year":{"$gte":1999}},{"$where":"this.offices.length >= 1"},
         {"$or":[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]}]}))

# Filtering and new collections

In [4]:
db, coll1 = f.connectCollection('companies','airports')
db, coll2 = f.connectCollection('companies','preferred')

In [5]:
airports = list(coll1.find({"type": 'large_airport'}))

In [6]:
business = list(coll2.find({"$and":[{"money_raised":{"$ne":"$0"}},{"year":{"$gte":2009}}]}))

Create new collections:

In [ ]:
'''
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["companies"]
mycol = mydb["final5"]
'''

To import a csv to Mongodb:

In [22]:
'''
mongoimport --db companies --collection kindergarten --type csv --file kindergarten.csv --headerline
'''

'\nmongoimport --db companies --collection kindergarten --type csv --file kindergarten.csv --headerline\n'

# Harversine 

Since the Harversine function takes a couple of coordinates and return the distance in between, it helped to conclude which of the 99 companies have an airport the closest. 

These 99 companies are the preferred after applying the different filters mentioned before. 

In [7]:
name=[]
for bus in business:
    name.append(bus['name'])

In [8]:
lat1=[]
long1=[]
f.getcoord(business,lat1,long1)

(99, 99)

In [9]:
bus=list(zip(lat1,long1))

In [10]:
lat2=[]
long2=[]
f.getcoord(airports,lat2,long2)

(614, 614)

In [11]:
air=list(zip(lat2,long2))

In [12]:
futcol=[]
for i in bus:
    futcol.append(f.harver(i,air))

In [13]:
dfair=pd.DataFrame({"name":name,"long":long1,"lat":lat1,"dis to an airport":futcol})

In [14]:
actuali=list(zip(business,futcol))

In [15]:
for i in actuali:
    value={"$set":{'distance':i[1]}}
    coll1.update_one(i[0],value)

In [16]:
cutoffs = [0,10,20,50,100,300]
km_labels = ['0-10km', '10-20km', '20-50km','50-100km','100-300km']
dfair['range'] = pd.cut(dfair['dis to an airport'], cutoffs, labels = km_labels)
dfair['range'].value_counts()

10-20km      43
0-10km       26
20-50km      24
50-100km      4
100-300km     2
Name: range, dtype: int64

# Google API

In [18]:
lat=[]
long=[]
f.getcoord(business,lat,long)

(99, 99)

In [19]:
lista=[]
busines=list(zip(lat,long))

In [20]:
for i in busines:
    lista.append("{},{}".format(i[0],i[1]))

### API request

In [24]:
start=time.time()
results=[]
for i in lista:    
    results.append(f.nearbyAPI(i, "2000", "nightclub"))
end=time.time()
print(end - start)

KeyboardInterrupt: 

In [ ]:
df_molona = f.createApiDf(results)

Be aware of dropping those entries which the API return duplicated.  

In [ ]:
df_molona=df_molona.drop_duplicates()

In [ ]:
#export_csv = df_molona.to_csv('nightclubs99.csv', index=None, header=True)

# Get Coord

In [25]:
pref_comp = list(coll.find())

In [26]:
for pref in pref_comp:
    value = {"$set": {'location':f.getApiLoc(pref)}}
    coll.update_one(pref,value)

KeyError: 'long'

# Geoqueries

In [30]:
db, coll3 = f.connectCollection('companies','starbucks')

In [31]:
star= list(coll3.find())

In [29]:
business = list(coll2.find({"$and":[{"money_raised":{"$ne":"$0"}},{"year":{"$gte":2009}}]}))

In [32]:
comparatorDosPuntoCero=[]
for bu in business:
    comparatorDosPuntoCero.append(f.geoquery(bu['location'],100,coll))

OperationFailure: error processing query: ns=companies.companiesTree: GEONEAR  field=location maxdist=100 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query

In [ ]:
cuantos=[]
for i in comparatorDosPuntoCero:
    cuantos.append(len(i))
data["starbucks 100m"]=cuantos

In [ ]:
#export_csv = data.to_csv('master99.csv', index=None, header=True)

In [ ]:
data = pd.read_csv("master99.csv") 
data.head()